# **First part : Data Collection and Data Cleaning**

## *Importing packages*

In [2]:
!pip install psaw
!pip install contractions
!pip install redditcleaner
!pip install nltk==3.4

     |████████████████████████████████| 284 kB 5.2 MB/s 
     |████████████████████████████████| 321 kB 43.1 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85442 sha256=3dae0a33af7550a609f0591ebbe477830164a245e1618d8270fe215e813b7763
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 1.4 MB 5.1 MB/s 
  Created wheel for nltk: filename=nltk-3.4-py3-none-any.whl size=1436397 sha256=5d637304bcab405fed1e6e50c5e7eec3d567244674908a6cf5d61c502a07437b
  Stored in directory: /root/.cache/pip/wheels/13/b8/81/2349be11dd144dc7b68ab983b58cd2fae353cdc50bbdeb09d0
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
from psaw import PushshiftAPI
import datetime
import pandas as pd
import contractions
import redditcleaner
from nltk.tokenize import word_tokenize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import nltk, time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
import collections, itertools

## *Data Collection Function*

In [ ]:
def get_reddit_posts(posted_before, posted_after, limit):
  api = PushshiftAPI()

  query = api.search_submissions(subreddit=['depression','Anxiety', 'bipolar'], filter=['selftext', 'title', 'subreddit'], before=posted_before, after=posted_after, limit=limit ) #add limit to test
  
  # Initiate empty list
  submissions = []

  # Write a for loop to fetch posts and append to our list
  print("\rFetching posts... fetched 0 posts.", end="")
  count = 0
  for ele in query:
      submissions.append(ele.d_)
      count += 1
      print(f"\rFetching posts... fetched {count} posts.", end="")

  # Print number of posts collected 
  print(len(submissions))

  df = pd.DataFrame(submissions)
  df['created_utc'] = df['created_utc'].apply(lambda x : datetime.datetime.fromtimestamp(x))
  
  return df

### From January 01 to Feb 28

In [ ]:
# posted_before = int(datetime.datetime(2021, 2, 28).timestamp())
# posted_after = int(datetime.datetime(2021, 1, 1).timestamp())
# df0 = get_reddit_posts(posted_before, posted_after, 90000)

### From Feb 01 to March 31

In [ ]:
# posted_before = int(datetime.datetime(2021, 3, 31).timestamp())
# posted_after = int(datetime.datetime(2021, 2, 1).timestamp())
# df1 = get_reddit_posts(posted_before, posted_after, 80000)

### From Apr 01 to May 31

In [ ]:
# posted_before = int(datetime.datetime(2021, 5, 31).timestamp())
# posted_after = int(datetime.datetime(2021, 4, 1).timestamp())
# df2 = get_reddit_posts(posted_before, posted_after, 75000)

### From Jun 01 to August 31

In [ ]:
# posted_before = int(datetime.datetime(2021, 8, 31).timestamp())
# posted_after = int(datetime.datetime(2021, 6, 1).timestamp())
# df3 = get_reddit_posts(posted_before, posted_after, 60000)

In [ ]:
df = pd.concat([df0, df1, df2, df3])

In [29]:
# df = pd.read_csv('/content/drive/MyDrive/Reddit Datasets/Mental-health-related-subreddits.csv')

In [49]:
df.sample(6)

,Title,Text,Subreddit
219470,I’m defeated,"I’m angry. I’m angry at myself, I’m angry at o...",depression
135901,I need somebody to talk to. Please.,I don't know where else to turn.\n\nI recently...,depression
189065,Breakup,Just wanted to check in. I've been struggling ...,depression
12980,Tips for talking to boyfriend about my anxiety?,Hi everyone!\nSo I had a bit of a anxiety atta...,Anxiety
443098,Just about to take my first dose of lamotrigen,I've had what was thought to be depression sin...,bipolar
77872,I don’t know what’s wrong with me,Hi I’m a 15 year old girl and for the last 2 y...,Anxiety


In [48]:
df.Subreddit.value_counts()

depression    180000
Anxiety        85000
bipolar        40000
Name: Subreddit, dtype: int64

In [50]:
df.shape

(305000, 3)

## *Basic Data Cleaning*

In [62]:
df.dropna(inplace = True)

In [52]:
# Rename selftext as text and subreddit as label 
# Create a dictionary with new column names
new_columns = {'Text': 'text', 'Title': 'title', 'Subreddit': 'label'} 

# Call rename () method
df.rename(columns=new_columns, inplace=True)

In [53]:
# Check number of duplicated rows
df.duplicated(subset=['text']).sum()

2043

In [54]:
# Duplicates removal
df.drop_duplicates(subset ="text",
                     keep = False, inplace = True)

In [55]:
# Our dataset has now 20363 posts
df.shape

(301747, 3)

In [ ]:
# Drop created_utc and created timestamps. 
# df.drop(['created_utc', 'created'], axis = 1, inplace=True)

In [56]:
# Number of submissions per subreddit 
df.groupby('label').count()

,title,text
label,,
Anxiety,84001,84001
bipolar,39292,39292
depression,178453,178454


In [57]:
# Merge selftext + title
df['text'] = df['text'] + ' ' + df['title']

In [58]:
# Title has been appended at the end of the post. 
df.head()

,title,text,label
124459,My thoughts,"in the end we all die, nothing we do means any...",depression
124460,When you're alone on New Years cause being wit...,Happy New Years guys When you're alone on New ...,depression
124461,It was never supposed to be like this,Today would have been my best friend's 18th bi...,depression
124462,Surge of confidence...but am I being delusiona...,"So I couldn't feel any worse, and I've had eno...",depression
124463,"Help with rejection, depression, suicide issues",I am 22 and have never had a girlfriend and ha...,depression


In [59]:
# Drop title column 
df.drop(['title'], axis = 1, inplace=True)
# df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [60]:
df.head()

,text,label
124459,"in the end we all die, nothing we do means any...",depression
124460,Happy New Years guys When you're alone on New ...,depression
124461,Today would have been my best friend's 18th bi...,depression
124462,"So I couldn't feel any worse, and I've had eno...",depression
124463,I am 22 and have never had a girlfriend and ha...,depression


In [63]:
# Check if there are any missing values 
# Output: False = No missing values
df.isnull().any()

text     False
label    False
dtype: bool

In [64]:
df.isna().any()

text     False
label    False
dtype: bool

In [65]:
df.shape

(301746, 2)

In [66]:
# Remove extra spaces 
df['text'] = df['text'].str.strip()

In [67]:
# Remove specific characters from Reddit markdown formatting with reddit cleaner 
df['text'] = df['text'].map(redditcleaner.clean)

In [68]:
# Remove emojis by encoding text as ascii while ignoring errors and decode as ascii.
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [69]:
df.head()

,text,label
124459,"in the end we all die, nothing we do means any...",depression
124460,Happy New Years guys When you're alone on New ...,depression
124461,Today would have been my best friend's 18th bi...,depression
124462,"So I couldn't feel any worse, and I've had eno...",depression
124463,I am 22 and have never had a girlfriend and ha...,depression


In [70]:
# Remove urls 
df['text'] = df['text'].str.replace('http\S+|www.\S+', '', case=False)

In [71]:
df.iloc[7]

text     EDIT: Wow that many upvotes and support, thank...
label                                           depression
Name: 124466, dtype: object

## *More Data Cleaning and preprocessing*

In [78]:
# Function to tokenize the Reddits
def custom_tokenize(text):
  """Function that tokenizes text"""
  if not text:
      print('The text to be tokenized is a None type. Defaulting to blank string.')
      text = ''
  return word_tokenize(str(text))

In [79]:
# Function that applies the cleaning steps
def clean_up(data):
  """Function that cleans up the data into a shape that can be further used for modeling"""
  # data.drop_duplicates() # drop duplicate reddits
  # data['text'].dropna(inplace=True) # drop any rows with missing reddits
  tokenized = data['text'].apply(custom_tokenize) # Tokenize reddits
  lower_tokens = tokenized.apply(lambda x: [t.lower() for t in x]) # Convert tokens into lower case
  alpha_only = lower_tokens.apply(lambda x: [t for t in x if t.isalpha()]) # Remove punctuations
  no_stops = alpha_only.apply(lambda x: [t for t in x if t not in stopwords.words('english')]) # remove stop words
  del data['text']
  data['text'] = no_stops
  return data

In [73]:
df_1 = df[df['label']=='Anxiety']
df_2 = df[df['label']=='bipolar']
df_3 = df[df['label']=='depression']

In [80]:
start = time.time()
# Read and clean the data
warnings.filterwarnings("ignore")
tidy_data_1 = clean_up(df_1)
tidy_data_2 = clean_up(df_2)
tidy_data_3 = clean_up(df_3)
end = time.time()
total = end-start
total

6951.411418914795

In [81]:
tidy_data = pd.concat([tidy_data_1, tidy_data_2, tidy_data_3,])
tidy_data.shape

(301746, 2)

In [83]:
tidy_data.head()

,label,text
0,Anxiety,"[struggled, social, anxiety, childhood, main, ..."
1,Anxiety,"[back, march, know, ago, woke, day, go, school..."
2,Anxiety,"[withdrawing, medicines, lately, effexor, zypr..."
3,Anxiety,"[first, time, poster, long, time, lurker, anxi..."
4,Anxiety,"[hi, made, throwaway, account, main, account, ..."


In [85]:
tidy_data.to_json('/content/drive/MyDrive/final_tidy_cleaned_data.json', orient='table')

## *Lemmatization*

In [2]:
path = '/content/drive/MyDrive/final_tidy_cleaned_data.json'
tidy_reddits = pd.read_json(path, orient='table')

In [3]:
tidy_reddits.shape

(301746, 2)

In [4]:
tidy_reddits.head()

,label,text
0,Anxiety,"[struggled, social, anxiety, childhood, main, ..."
1,Anxiety,"[back, march, know, ago, woke, day, go, school..."
2,Anxiety,"[withdrawing, medicines, lately, effexor, zypr..."
3,Anxiety,"[first, time, poster, long, time, lurker, anxi..."
4,Anxiety,"[hi, made, throwaway, account, main, account, ..."


In [5]:
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [6]:
# Lemmatize tokens
start = time.time()

lemmatizer = WordNetLemmatizer()
tidy_reddits['lemmatized'] = tidy_reddits['text'].apply(lambda x: [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in x])

end = time.time()
total = end-start
total

4293.25057721138

In [7]:
tidy_reddits['tokens_back_to_text'] = [' '.join(map(str, l)) for l in tidy_reddits['lemmatized']]

In [8]:
tidy_reddits['tokens_back_to_text'][3]

'first time poster long time lurker anxiety year always cripple miserable experience feel like lose much life due scar pursue friendship past hello self conscious voice opinion still hard still ca fully around many many people struggle hold conversation without feel like burden recently make proud thought share finish school year month freedom uni say accomplish goal never thought would one big goal complete interview something really want usually would even attempt fear put fear aside hate phone call almost always ignore yet able one one twenty minute interview though felt uncomfortable talk stranger fact feel amaze hearing later accepted one best feeling ever felt like wall surround slowly chip away small scale push text people first even talk age felt like hat prove hard initially felt like idiot anxiety push million negative thought head regardless anxiety thought push wall anxiety proud talk so family anxiety make feel like absolutely despise result neglect talk time go felt disco

In [9]:
# Explort the lemmatized data to json file for ease of use in the next step
path = '/content/drive/MyDrive/final_tidy_reddits_lemmetized.json'
tidy_reddits.to_json(path, orient='table')

In [10]:
tidy_reddits.shape

(301746, 4)

In [14]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [15]:
tidy_reddits.sample(20)

,label,text,lemmatized,tokens_back_to_text
439663,bipolar,"[either, really, btchy, mood, mood, everything, fine, couple, years, increasing, fixated, house, fixates, self, worth, nice, house, moved, times, last, years, time, laments, sold, gets, moods, everything, horrible, starts, blaming, everyone, unhappiness, never, much, threatens, move, us, break, marriage, said, multiple, times, also, constantly, yell, yet, sometimes, fine, loving, normal, family, worried, puzzled, help, know, thyroid, wack, takes, naturopathic, pills, morning, never, excercises, except, work, nurse, shift, work, fix, would, really, appreciated, think, moms, bipolar, tearing, family, apart, advice]","[either, really, btchy, mood, mood, everything, fine, couple, year, increase, fixate, house, fixates, self, worth, nice, house, move, time, last, year, time, lament, sell, get, mood, everything, horrible, start, blame, everyone, unhappiness, never, much, threatens, move, u, break, marriage, say, multiple, time, also, constantly, yell, yet, sometimes, fine, love, normal, family, worried, puzzle, help, know, thyroid, wack, take, naturopathic, pill, morning, never, excercises, except, work, nurse, shift, work, fix, would, really, appreciate, think, mom, bipolar, tear, family, apart, advice]",either really btchy mood mood everything fine couple year increase fixate house fixates self worth nice house move time last year time lament sell get mood everything horrible start blame everyone unhappiness never much threatens move u break marriage say multiple time also constantly yell yet sometimes fine love normal family worried puzzle help know thyroid wack take naturopathic pill morning never excercises except work nurse shift work fix would really appreciate think mom bipolar tear family apart advice
268300,depression,"[graduated, high, school, decided, take, year, worked, bit, end, december, decided, move, big, city, take, acting, courses, really, enjoyed, couple, months, decided, look, job, staying, little, bit, planning, staying, upon, looking, getting, job, immediately, hated, triggered, something, felt, sad, working, quit, shifts, thought, would, go, away, finished, courses, moved, back, home, process, looking, job, pay, university, think, depression, getting, worse, makes, fucking, sad, work, job, know, hate, makes, feel, worthless, unaccomplished, want, thing, enjoy, time, school, ca, friends, hardly, talk, anyone, besides, family, want, someone, hang, talk, life, fun, things, together, excited, university, second, chance, go, make, friends, join, clubs, ...]","[graduate, high, school, decide, take, year, work, bit, end, december, decide, move, big, city, take, act, course, really, enjoy, couple, month, decide, look, job, stay, little, bit, planning, stay, upon, look, get, job, immediately, hat, trigger, something, felt, sad, work, quit, shift, thought, would, go, away, finish, course, move, back, home, process, look, job, pay, university, think, depression, get, bad, make, fuck, sad, work, job, know, hate, make, feel, worthless, unaccomplished, want, thing, enjoy, time, school, ca, friend, hardly, talk, anyone, besides, family, want, someone, hang, talk, life, fun, thing, together, excite, university, second, chance, go, make, friend, join, club, ...]",graduate high school decide take year work bit end december decide move big city take act course really enjoy couple month decide look job stay little bit planning stay upon look get job immediately hat trigger something felt sad work quit shift thought would go away finish course move back home process look job pay university think depression get bad make fuck sad work job know hate make feel worthless unaccomplished want thing enjoy time school ca friend hardly talk anyone besides family want someone hang talk life fun thing together excite university second chance go make friend join club learn thing actually interested time till get finally accepted depressed
245123,depression,"[basically, wan, na, know, 